In [1]:
import pandas as pd
import json
from rich import print

In [2]:
augmentation_path = '../../data/data_augmentation/gpt_augmentation/final_augmentation_full_data_5_examples.json'
with open(augmentation_path) as f:
    augmentation_data_text = json.load(f)

In [3]:
# data = pd.read_csv('../../data/cleansed_data/topics/first_iteration/label_issues_topics_majority_vote_first_iteration_logistic_regression(solved).csv')
data = pd.read_csv('../../data/result/train_with_topic_df_majority_vote_gpt4o_preferred.csv')

In [4]:
data.head()

,column,text,topic_id,topic_name,topic_llm,majority_vote,tie
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة,طريقة تدريس و تقديم المادة,False
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False


In [5]:
data = data.drop(columns=['column', 'topic_id', 'topic_name', 'topic_llm', 'tie'])
data = data.rename(columns={'majority_vote': 'topic_name'})
data.head()

,text,topic_name
0,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,موازنة الجزء العملي مع الجزء النظري
1,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,طريقة تدريس و تقديم المادة
2,أكثر ما أعجبني: the content,محتوى ومعلومات المقرر
3,اقتراحاتي للتحسين: provide more new books,محتوى ومعلومات المقرر
4,اقتراحاتي للتحسين: زيادة ساعات العملي,موازنة الجزء العملي مع الجزء النظري


In [6]:
def augment_one_class(data, augmentation_data, class_name, labels_col, augmentation_col, target_length, duplicate=False):
    class_data = list(data[data[labels_col]==class_name][augmentation_col])
    
    if len(class_data) == 0:
        raise ValueError(f"Class '{class_name}' not found in column '{labels_col}'")
    
    augmented = []
    
    num_augmentation_examples = len(list(augmentation_data.values())[0])
    
    flag = False
    for i in range(num_augmentation_examples):
        for example in class_data:
            if len(class_data + augmented) >= target_length:
                flag = True
                break
            augmented.append(list(augmentation_data[example].values())[i])
        
        if flag:
            break
    
    if duplicate:
        if len(class_data + augmented) < target_length:
            for example in class_data:
                augmented.append(example)
                if len(class_data + augmented) >= target_length:
                    break
        
            
            for example in augmented.copy():
                if len(class_data + augmented) >= target_length:
                    break
                
                augmented.append(example)
                
    augmented_df = pd.DataFrame()
    augmented_df[augmentation_col] = augmented
    augmented_df[labels_col] = class_name
    
    return pd.concat([data, augmented_df], ignore_index=True)

In [7]:
data.topic_name.value_counts()

محتوى ومعلومات المقرر                  226
دكتور المقرر                           176
موازنة الجزء العملي مع الجزء النظري    166
غير محدد                               131
المتطلبات و المهام و الدرجات            74
طريقة تدريس و تقديم المادة              73
الوقت و الجدول                          39
الجانب البحثي و الأوراق البحثية         35
سوق العمل                               31
Name: topic_name, dtype: int64

In [8]:
len(data)

951

In [9]:
minority_classes = [
    "طريقة تدريس و تقديم المادة",
    "المتطلبات و المهام و الدرجات",
    "الوقت و الجدول",
    "الجانب البحثي و الأوراق البحثية",
    "سوق العمل"
]

In [10]:
minority = False
duplicate = False
target_length = 222

In [11]:
if minority:
    classes_to_augment = minority_classes
else:
    classes_to_augment = list(data['topic_name'].unique())

for class_name in classes_to_augment:
    
    data = augment_one_class(
        data=data, augmentation_data=augmentation_data_text, class_name=class_name, labels_col='topic_name',
        augmentation_col='text',target_length=target_length, duplicate=duplicate
    )

In [12]:
len(data)

1954

In [13]:
data.topic_name.value_counts()

محتوى ومعلومات المقرر                  226
موازنة الجزء العملي مع الجزء النظري    222
طريقة تدريس و تقديم المادة             222
دكتور المقرر                           222
غير محدد                               222
المتطلبات و المهام و الدرجات           222
الوقت و الجدول                         222
الجانب البحثي و الأوراق البحثية        210
سوق العمل                              186
Name: topic_name, dtype: int64

In [14]:
data.head()

,text,topic_name
0,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,موازنة الجزء العملي مع الجزء النظري
1,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,طريقة تدريس و تقديم المادة
2,أكثر ما أعجبني: the content,محتوى ومعلومات المقرر
3,اقتراحاتي للتحسين: provide more new books,محتوى ومعلومات المقرر
4,اقتراحاتي للتحسين: زيادة ساعات العملي,موازنة الجزء العملي مع الجزء النظري


In [15]:
random_state = 42

In [16]:
data = data.sample(frac=1.0, random_state=random_state)

# convert to hf dataset

In [17]:
from datasets import Dataset, Value, ClassLabel, Features, DatasetDict, load_dataset, load_from_disk

In [18]:
topic_names = sorted(data['topic_name'].unique().tolist())
topic_names

['الجانب البحثي و الأوراق البحثية',
 'المتطلبات و المهام و الدرجات',
 'الوقت و الجدول',
 'دكتور المقرر',
 'سوق العمل',
 'طريقة تدريس و تقديم المادة',
 'غير محدد',
 'محتوى ومعلومات المقرر',
 'موازنة الجزء العملي مع الجزء النظري']

In [19]:
features = Features(
    {
        'id': Value(dtype='string'),
        'text': Value(dtype='string'),
        'labels': ClassLabel(names=topic_names, num_classes=len(topic_names), id=None)
    }
)

In [20]:
print(features)

{
    'id': Value(dtype='string', id=None),
    'text': Value(dtype='string', id=None),
    'labels': ClassLabel(
        names=[
            'الجانب البحثي و الأوراق البحثية',
            'المتطلبات و المهام و الدرجات',
            'الوقت و الجدول',
            'دكتور المقرر',
            'سوق العمل',
            'طريقة تدريس و تقديم المادة',
            'غير محدد',
            'محتوى ومعلومات المقرر',
            'موازنة الجزء العملي مع الجزء النظري'
        ],
        id=None
    )
}

In [21]:
hf_dataset = DatasetDict({
    'train': Dataset.from_dict(
        {
            'id': list(data.index.astype(str)),
            'text': list(data['text']),
            'labels': list(data['topic_name'])
        },
        features=features
    )
})

In [22]:
hf_path = '../../data/result/final_topic_hf_dataseet_full_augmentation_balanced1'

In [23]:
hf_dataset.save_to_disk(hf_path)

Saving the dataset (0/1 shards):   0%|          | 0/1954 [00:00<?, ? examples/s]